In [1]:
import numpy as np
import math
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp

In [2]:
j = 12
n = 10
a = 0.7
b = 1.7
alpha_j = 0.1 + 0.05*j
h = 1/n

def f(x):
    return alpha_j * math.exp(x) + (1 - alpha_j) * math.sin(x)

In [8]:
x_vals = np.array(sorted([(a+b) / 2 + (b-a) / 2 * math.cos(math.pi * (2*i+1) / (2*n+1)) for i in range(n+1)]))
f_vals = np.array([f(x_) for x_ in x_vals])
x_star = np.array([x_vals[0] + 2/3*h, x_vals[n // 2] / 2 + 0.5 * h, x_vals[-1] - 1/3*h])
f_star = np.array([f(x_) for x_ in x_star])

print(tabulate(zip(x_vals, f_vals), headers=['x', 'f(x)']))
print('\nСпециальные точки')
print(tabulate(zip(x_star, f_star), headers=['x*', 'f(x*)']))

       x     f(x)
--------  -------
0.7       1.60289
0.715708  1.62879
0.761847  1.70664
0.835516  1.83669
0.932087  2.01872
1.04549   2.25091
1.1686    2.52831
1.29369   2.84091
1.41289   3.17173
1.51871   3.49603
1.60451   3.78262
1.66489   3.99823
1.69606   4.11434

Специальные точки
      x*    f(x*)
--------  -------
0.766667  1.71493
0.634302  1.49778
1.66272   3.99029


In [9]:
# Таблица значений функции
table = pd.DataFrame({"x_i": x_vals, "f(x_i)": f_vals})
table_transposed = table.T

# Точки для проверки интерполяции
x_star1 = x_star[0]
x_star2 = x_star[1]
x_star3 = x_star[2]

f_x_star = f_star[0]
f_x_star2 = f_star[1]
f_x_star3 = f_star[2]

def compute_newton_coefficients(x_vals, y_vals):
    """
    Возвращает список коэффициентов интерполяционного многочлена Ньютона
    с использованием рекурсивного определения разделённых разностей.
    """
    n = len(x_vals)
    # Создаём таблицу размером n x n
    dd_table = [y_vals.copy()]  # f[x_i]

    for level in range(1, n):
        prev_column = dd_table[-1]
        curr_column = []
        for i in range(n - level):
            numerator = prev_column[i + 1] - prev_column[i]
            denominator = x_vals[i + level] - x_vals[i]
            curr_column.append(numerator / denominator)
        dd_table.append(curr_column)

    # Коэффициенты Ньютона — это верхние элементы каждого столбца
    return dd_table, [dd_table[i][0] for i in range(n)]
    
dd_table, newton_coeffs = compute_newton_coefficients(x_vals, f_vals)
def newton_interpolation(x_vals, y_vals, x, coef):
    """
    Вычисляет значение интерполяционного многочлена Ньютона в точке x
    с использованием рекурсивной формулы:
    P_{n+1}(x) = P_n(x) + alpha_{n+1} * omega_{n+1}(x)
    """
    result = coef[0]
    omega = 1.0
    for i in range(1, len(coef)):
        omega *= (x - x_vals[i - 1])
        result += coef[i] * omega
    return result, omega

omegas = [1.0,1.0,1.0]
P_x_star, omegas[0] = newton_interpolation(x_vals, f_vals, x_star1, newton_coeffs)
P_x_star2, omegas[1] = newton_interpolation(x_vals, f_vals, x_star2, newton_coeffs)
P_x_star3, omegas[2] = newton_interpolation(x_vals, f_vals, x_star3, newton_coeffs)

omega_frame = pd.DataFrame(omegas, index=[f'omega_{i}' for i in range(len(omegas))])
# Результаты интерполяции
data = {
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star1, x_star2, x_star3],
    "f(x)": [f_x_star, f_x_star2, f_x_star3],
    "P(x) (полином)": [P_x_star, P_x_star2, P_x_star3]
}

n = len(newton_coeffs)
dd_frame = pd.DataFrame(dd_table, columns=[f"f[x0..x{i}]" for i in range(len(newton_coeffs))])
dd_frame.insert(0, "x_i", x_vals)
coeff_frame = pd.DataFrame(newton_coeffs, index=[f"a_{i}" for i in range(n)]).T
df = pd.DataFrame(data)

# Производная (n+1)-го порядка
x = sp.Symbol('x')
f_sym = alpha_j * sp.exp(x) + (1 - alpha_j) * sp.sin(x)
f_derivative = sp.diff(f_sym, x, n + 1)

# Максимум абсолютного значения производной на отрезке [0.7, 1.7]
f_derivative_abs = sp.lambdify(x, sp.Abs(f_derivative), 'numpy')
x_test = np.linspace(0.7, 1.7, 1000)
M_max = np.max(f_derivative_abs(x_test))

# Истинная погрешность
r_x_star = f_x_star - P_x_star
r_x_star2 = f_x_star2 - P_x_star2
r_x_star3 = f_x_star3 - P_x_star3

# Оценка погрешности по неравенству
factorial = math.factorial(n + 1)
x_stars = [x_star1, x_star2, x_star3]
r_x_stars = [r_x_star, r_x_star2, r_x_star3]
error_bound_stars = []

for x_val in x_stars:
    prod_term = np.prod([abs(x_val - xi) for xi in x_vals])
    error_bound = M_max / factorial * prod_term
    error_bound_stars.append(error_bound)

# Проверка выполнения неравенства
is_error_bound_stars_valid = [
    abs(r_x_stars[i]) <= error_bound_stars[i] for i in range(3)
]

# Таблица ошибок
error_table = pd.DataFrame({
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star1, x_star2, x_star3],
    "r истинная": [abs(r) for r in r_x_stars],
    "оценка погрешности": error_bound_stars,
    "M = max|f^(n+1)(x)|": [M_max] * 3,
    "Неравенство выполняется?": is_error_bound_stars_valid
})

# Вывод таблиц
display(table_transposed)
display(df)
display(error_table)
display(dd_frame)
display(coeff_frame)
display(omega_frame)




,0,1,2,3,4,5,6,7,8,9,10,11,12
x_i,0.700000,0.715708,0.761847,0.835516,0.932087,1.045492,1.168605,1.293691,1.412890,1.518712,1.604508,1.664888,1.696057
f(x_i),1.602892,1.628790,1.706638,1.836695,2.018722,2.250909,2.528312,2.840908,3.171733,3.496031,3.782619,3.998231,4.114335


,Точка,Значение x,f(x),P(x) (полином)
0,x*,0.766667,1.714927,1.714927
1,x**,0.634302,1.497779,1.497779
2,x***,1.662724,3.990293,3.990293


,Точка,Значение x,r истинная,оценка погрешности,M = max|f^(n+1)(x)|,Неравенство выполняется?
0,x*,0.766667,4.440892e-16,1.517883e-19,3.534264,False
1,x**,0.634302,2.109424e-14,1.754768e-16,3.534264,False
2,x***,1.662724,8.881784e-16,3.422546e-19,3.534264,False


,x_i,f[x0..x0],f[x0..x1],f[x0..x2],f[x0..x3],f[x0..x4],f[x0..x5],f[x0..x6],f[x0..x7],f[x0..x8],f[x0..x9],f[x0..x10],f[x0..x11],f[x0..x12]
0,0.700000,1.602892e+00,1.628790e+00,1.706638e+00,1.836695,2.018722,2.250909,2.528312,2.840908,3.171733,3.496031,3.782619,3.998231,4.114335
1,0.715708,1.648682e+00,1.687260e+00,1.765426e+00,1.884906,2.047415,2.253234,2.499056,2.775401,3.064548,3.340322,3.570928,3.724982,NaN
2,0.761847,6.237767e-01,6.524244e-01,7.018377e-01,0.773939,0.870205,0.990421,1.131241,1.284977,1.439177,1.577594,1.682746,NaN,NaN
3,0.835516,2.113978e-01,2.283653e-01,2.541955e-01,0.289011,0.332450,0.383291,0.439110,0.496113,0.549275,0.592921,NaN,NaN,NaN
4,0.932087,7.310840e-02,7.832472e-02,8.559286e-02,0.094809,0.105741,0.117956,0.130770,0.143219,0.154132,NaN,NaN,NaN,NaN
5,1.045492,1.509824e-02,1.604814e-02,1.732888e-02,0.018933,0.020824,0.022921,0.025085,0.027124,NaN,NaN,NaN,NaN,NaN
6,1.168605,2.027089e-03,2.215880e-03,2.463831e-03,0.002768,0.003118,0.003494,0.003865,NaN,NaN,NaN,NaN,NaN,NaN
7,1.293691,3.179954e-04,3.556481e-04,4.019310e-04,0.000455,0.000513,0.000571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.412890,5.281698e-05,5.763734e-05,6.327429e-05,0.000069,0.000076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.518712,5.887740e-06,6.342204e-06,6.867258e-06,0.000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,a_11,a_12
0,1.602892,1.648682,0.623777,0.211398,0.073108,0.015098,0.002027,0.000318,0.000053,0.000006,5.024433e-07,5.256792e-08,4.579481e-09


,0
omega_0,-4.028556e-09
omega_1,4.076662e-06
omega_2,-2.532677e-07
